In [1]:
import readability
import json
import pandas as pd

In [4]:
#Create a dataframe with all stories and their generations

all_stories = json.load(open('../books/real_processed.json'))
all_stories = pd.DataFrame(all_stories)
all_stories.head()
gen_stories = json.load(open('../books/from_server/gen_stories_opt.json'))
gen_stories = pd.DataFrame(gen_stories)
all_stories = all_stories.merge(gen_stories, on=['id'], how='left')

all_stories['story_len'] = 0
all_stories['gen_story_len'] = 0
all_stories['gen_story_only_len'] = 0
all_stories['story_score'] = 0
all_stories['gen_story_score'] = 0
all_stories['gen_story_only_score'] = 0
all_stories.head()

,id,text,title,gen_id,p_length,model_name,gen_text,prompt,story_len,gen_story_len,gen_story_only_len,story_score,gen_story_score,gen_story_only_score
0,0,It was beautiful in the country. It was summer...,THE UGLY DUCKLING\n,0,first_line,opt,It was beautiful in the country. It rained on...,It was beautiful in the country.,0,0,0,0,0,0
1,0,It was beautiful in the country. It was summer...,THE UGLY DUCKLING\n,1,first_line,opt,It was beautiful in the country.\nIt was the m...,It was beautiful in the country.,0,0,0,0,0,0
2,0,It was beautiful in the country. It was summer...,THE UGLY DUCKLING\n,2,first_line,opt,It was beautiful in the country. Really green ...,It was beautiful in the country.,0,0,0,0,0,0
3,0,It was beautiful in the country. It was summer...,THE UGLY DUCKLING\n,3,256_tokens,opt,It was beautiful in the country. It was summer...,</s>It was beautiful in the country. It was su...,0,0,0,0,0,0
4,0,It was beautiful in the country. It was summer...,THE UGLY DUCKLING\n,4,256_tokens,opt,It was beautiful in the country. It was summer...,</s>It was beautiful in the country. It was su...,0,0,0,0,0,0


In [5]:
for i, story in all_stories.iterrows():
    s = story['text']
    s_gen = story['gen_text']
    s_gen_only = story['gen_text'].replace(story['prompt'], '')
    all_stories.loc[i,'story_len'] = len(s.split())
    all_stories.loc[i,'gen_story_len'] = len(s_gen.split())
    all_stories.loc[i,'gen_story_only_len'] = len(s_gen_only.split())
    all_stories.loc[i,'story_score'] = readability.getmeasures(s, lang='en')['readability grades']['FleschReadingEase']
    all_stories.loc[i,'gen_story_score'] = readability.getmeasures(s_gen, lang='en')['readability grades']['FleschReadingEase']
    all_stories.loc[i,'gen_story_only_score'] = readability.getmeasures(s_gen_only, lang='en')['readability grades']['FleschReadingEase']

In [6]:
all_stories.groupby(by=['p_length']).mean()

/var/folders/hm/fpq61bcd7vvgs4q8dtq8qn6m0000gp/T/ipykernel_21008/3087029325.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_stories.groupby(by=['p_length']).mean()


,id,gen_id,story_len,gen_story_len,gen_story_only_len,story_score,gen_story_score,gen_story_only_score
p_length,,,,,,,,
256_tokens,0.5,4.0,3259.0,724.000000,724.000000,97.609144,100.070886,100.070886
512_tokens,0.5,7.0,3259.0,711.833333,711.833333,97.609144,100.015844,100.015844
first_line,0.5,1.0,3259.0,539.500000,517.500000,97.609144,95.725250,97.792520
